### Build a Simple LLM Application with LCEL
This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [6]:
### Use open source models -- Llama3, Gemma2, mistral
# Using Groq

import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
print(groq_api_key)

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


In [7]:
pip install langchain_groq

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain_groq]m [langchain_groq]
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-it", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10cab8e60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10cab9fa0>, model_name='Gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
pip install langchain_core

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content = "Translate the following from English to French"),
    HumanMessage(content="Hello, how are you?")

]

result = model.invoke(messages)
result

AIMessage(content='Bonjour, comment allez-vous ? \n\n\nYou can also use these variations:\n\n* **Bonjour, ça va ?** (Informal)\n* **Salut, comment vas-tu ?** (Very informal)\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 22, 'total_tokens': 68, 'completion_time': 0.083636364, 'prompt_time': 0.002149876, 'queue_time': 0.066492484, 'total_time': 0.08578624}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--cfd1e5e1-bcf7-4568-9f73-006be6a9b656-0', usage_metadata={'input_tokens': 22, 'output_tokens': 46, 'total_tokens': 68})

In [13]:
#string output parser to retrieve just the output
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser() #just display content message
parser.invoke(result) #take the content from AI message and display

"Bonjour, comment allez-vous ? \n\n\nLet me know if you'd like to translate anything else!\n"

Using LCEL - chain the components

In [15]:
chain = model|parser
chain.invoke(messages) #send messages to the model, model will give output, output will go to parser to give output

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, ça va ? \n'

In [16]:
### Prompt templates
from langchain_core.prompts import ChatPromptTemplate

#define generic template
generic_template = "Translate the following into {language}:"
prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")] #we have to give text at runtime
    
)

In [20]:
resultNew = prompt.invoke({"language":"French", "text":"Hello"})

In [21]:
resultNew.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [22]:
chain = prompt|model|parser #using LCEL, we have combined prompt, model, parser
chain.invoke({"language":"French", "text":"Hello"})

'Bonjour \n'